In [39]:
from ipyleaflet import *
from ipywidgets import *
from random import *

import pandas as pd
import numpy as np
import math

import xarray as xr
import os
import scipy

import pyqtree as qt
import bbox as bb

from collections import *
import operator
import time

In [76]:
class mappDataFrame():
    
    def __init__(self, dataType = [], dataDate = None, airport = None):
    
        self.dataType = []
        self.dataDate = None
        self.airport = None
        self.adsbDate = None
        self.adsbDateHyphen = None
        self.tfms = None
        self.adsbDf = None
        self.tfmsDf = None
        self.aeroDf = None
        self.uasFlights = None
        self.adiHigh = None
        self.adiLow = None
        self.timeOut = []
        self.adsbFlightsDictionary = {}
        self.adsbFlights = None
        self.adsbLines = None
        self.adsbDisplay = None
        self.adsbHeat = {}
        self.uasOd = None
        self.uasLines = None
        self.uasDisplay = None
        self.uasHeat = {}
        self.heatDistanceDelimiter = None
        self.encounterResults = None
        self.encounterFlights = None
        self.heatCircles = None
        self.heatDisplay = None
        self.mapp = None
        
    def buildDataFrames(self, dataType, dataDate, airport = None):
        
        for dataTypes in dataType:
            self.dataType.append(dataTypes)
        
        if 'ADSB' in self.dataType:
            adsbDate = dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')
            
        
        if 'TFMS' in self.dataType:
            
            self.airport = airport
            self.tfmsDf = pd.read_csv('./TFMS/'+ self.airport + ".csv")
            
        if 'AERO' in self.dataType:
            
            xls = pd.ExcelFile('./AERO/aero8-14.xlsx')
            sheet_to_df_map = {}
            for sheet_name in xls.sheet_names:
                sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
                sheet_to_df_map[sheet_name] = sheet_to_df_map[sheet_name].sort_values(['TimeWhen'])
                
            uasFlights = {}
            for df in sheet_to_df_map:

                uasFlights[str(sheet_to_df_map[df]['Drone ID'][0])] =[ [sheet_to_df_map[df]['Latitude'],
                                                                       sheet_to_df_map[df]['Longitude']],
                                                                      sheet_to_df_map[df]['TimeWhen'],
                                                                      sheet_to_df_map[df]['IconAltitude']]
            self.uasFlights = uasFlights

            
        
    def rebuildAdsbDf(self):
        
        if 'ADSB' in self.dataType:
            adsbDate = self.dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')

        
    def refreshParameters(self, adiLow, adiHigh, timeOut):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh
        self.timeOut = timeOut
        
    def refreshAltitude(self, adiLow, adiHigh):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh

    def refreshTime(self, timeOut):
        self.timeOut = timeOut
                    
    def refreshDate(self, date):
        self.dataDate = date
        
    def RGBtoHex(self, RGB):
        # Components need to be integers for hex to make sense
        RGB = [int(x) for x in RGB]
        return "#"+"".join(["0{0:x}".format(v) if v < 16 else
                "{0:x}".format(v) for v in RGB])


    def buildAdsbFlights(self):
        

        flights = []
        flights.extend({self.adsbDf.iat[0,11]})

        for y in range(len(self.adsbDf)):

            acidCur = self.adsbDf.iat[y - 1 , 11]

            if(self.adsbDf.iat[y, 11] != acidCur ):

                if(acidCur in flights):
                    ;
                else:
                    flights.extend([acidCur])

        for i in flights:

            if pd.isnull(i) == True:
                flights.remove(i)

        self.adsbFlights = flights
        
    def refreshAdsbFlights(self):

        adsbFlightsDictionary = {}
        adiLow = self.adiLow
        adiHigh = self.adiHigh
        timeOut = self.timeOut

        for items in self.adsbFlights:

            if type(items) == str:

                adsbFlightsDictionary.update({items+'track':[]})
                adsbFlightsDictionary.update({items+'time':[]})
                adsbFlightsDictionary.update({items+'altitude':[]})  
                
        for x in range(len(self.adsbDf)):

            timeIn = self.adsbDf.iat[x,1]
            timeIn = time.strftime('%H:%M:%S', time.gmtime(timeIn))
            timeIn = ' '.join([self.adsbDateHyphen, timeIn])
            
            if type(self.adsbDf.iat[x,11]) == str:
                
                
                if timeIn >= self.adsbDateHyphen + timeOut[0] and timeIn <= self.adsbDateHyphen + timeOut[1]:
                    if int(self.adsbDf.iat[x,7]) >= adiLow and int(self.adsbDf.iat[x,7]) <= adiHigh:
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "track"].append([self.adsbDf.iat[x,4] , self.adsbDf.iat[x,5]])    
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "time"].append(timeIn)
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "altitude"].append([self.adsbDf.iat[x,10], self.adsbDf.iat[x,7]])
                        
        self.adsbFlightsDictionary = adsbFlightsDictionary

    def refreshAdsbDisplay(self):
        
        lp = 0
        lines = []
        colorRange = 255/len(self.adsbFlights)
        
        for items in self.adsbFlights:
            if lp == 0:
                colorIn = self.RGBtoHex([0,0,255])
            else:
                colorIn = self.RGBtoHex([ 0, int(255-(colorRange * lp)), 255 ])
            
            flightName = items
            
            items = Polyline(
            locations = 
                [[
                    self.adsbFlightsDictionary[items + 'track']
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            if(type(self.adsbFlights[lp]) == str):
                items.popup=HTML(flightName)

            lines.append(items)
            lp+=1;
            
            
        self.adsbLines = lines
        if self.adsbDisplay == None:
            self.adsbDisplay = LayerGroup(layers=(lines)) 
        else:
            self.adsbDisplay.layers = self.adsbLines
            
            
    def refreshUasFlights(self):
        flightTracks = []
        flightTimes = []
        flightAltitudes = []
        uasOd = {}

        timeMin = self.adsbDateHyphen + self.timeOut[0]
        timeMax = self.adsbDateHyphen + self.timeOut[1]
        adiLow = self.adiLow
        adiHigh = self.adiHigh

        for z in self.uasFlights:
            tracks = []
            times = []
            altitudes = []
            heatIterator = 0
            for i in range(len(self.uasFlights[z][0][0])):
                k = self.uasFlights[z][0][0][i]
                j = self.uasFlights[z][0][1][i]
                time = self.uasFlights[z][1][i]
                altitude = self.uasFlights[z][2][i]
                if type(time) == str:
                    if time >= timeMin and time <= timeMax:
                         if altitude >= adiLow and altitude <= adiHigh:
                            tracks.append([k,j])
                            times.append(time)
                            altitudes.append(altitude)
                            self.uasHeat.update({z + str(heatIterator):[]})
                            heatIterator+=1
                            
                            

            flightTimes.append(times)
            flightTracks.append(tracks)
            flightAltitudes.append(altitudes)
            uasOd[z] = (flightTimes, flightTracks,flightAltitudes)
            self.uasOd = uasOd

            
    def refreshUasDisplay(self):
        
        
        lp = 0
        lines = []
        colorRange = 255/len(self.uasFlights)

        for i in self.uasFlights:
            if lp == 0:
                colorIn = self.RGBtoHex([255,0,0])
            else:
                colorIn = self.RGBtoHex([ 255 , 0, int(255-(colorRange * lp)) ])

            
            line = Polyline(
            locations = 
                [[
                    self.uasOd[i][1][lp]
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            line.popup=HTML(i)
            lines.append(line)


            lp+=1;
            
        self.uasLines = lines
        if self.uasDisplay == None:
            self.uasDisplay = LayerGroup(layers=(lines)) 
        else:
            self.uasDisplay.layers = self.uasLines
             
    def refreshUasHeatTree(self, heatDistanceDelimiter):
        
        
        self.heatDistanceDelimiter = heatDistanceDelimiter
        
        if self.heatDisplay == None:
            circles = []

            iterator = 0
            uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
            for uasFlight in self.uasFlights:
                uasTreeCoordIndex = 0
                for coordinate in self.uasOd[uasFlight][1][iterator]:

                    latitudeFeetConversion = 6076 * 60
                    
                    xmin = ((coordinate[0] * 6076)*60 - self.heatDistanceDelimiter) / latitudeFeetConversion
                    xmax = ((coordinate[0] * 6076)*60 + self.heatDistanceDelimiter) / latitudeFeetConversion
                    ymin = ((coordinate[1] * 6076)*60 - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / latitudeFeetConversion
                    ymax = ((coordinate[1] * 6076)*60 + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / latitudeFeetConversion
                    self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)

                    uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                    uasTreeCoordIndex+=1

                    heatCircle = Circle()
                    heatCircle.location = coordinate
                    heatCircle.radius = int(self.heatDistanceDelimiter / 3)
                    heatCircle.popup = HTML(uasFlight + '\n waypt#' +str(uasTreeCoordIndex))
                    heatCircle.color = 'darkred'
                    heatCircle.opacity = .3
                    heatCircle.weight = 1
                    heatCircle.fill_color='darkred'
                    circles.append(heatCircle)

                iterator+=1

            self.heatCircles = circles
            self.uasTree = uasTree
            self.heatDisplay = LayerGroup(layers=(circles)) 
        else:
            iterator = 0
            uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
            for uasFlight in self.uasFlights:
                uasTreeCoordIndex = 0
                for coordinate in self.uasOd[uasFlight][1][iterator]:

                    latitudeFeetConversion = 6076 * 60
                    
                    xmin = ((coordinate[0] * 6076)*60 - self.heatDistanceDelimiter) / latitudeFeetConversion
                    xmax = ((coordinate[0] * 6076)*60 + self.heatDistanceDelimiter) / latitudeFeetConversion
                    ymin = ((coordinate[1] * 6076)*60 - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / latitudeFeetConversion
                    ymax = ((coordinate[1] * 6076)*60 + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / latitudeFeetConversion
                    self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)

                    uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                    uasTreeCoordIndex+=1

                iterator+=1
            self.uasTree = uasTree  
            for heatCircle in self.heatCircles:
                heatCircle.radius = int(self.heatDistanceDelimiter / 3.281)
            self.heatDisplay.layers = self.heatCircles
        ;
        
        
    def refreshAdsbTree(self):
        

        adsbTree = qt.Index(bbox= (-90, -180, 90, 180))

        for flight in self.adsbFlights:
            flightTreeIndex = 0
            for coordinates in self.adsbFlightsDictionary[flight + 'track']:
                adsbTree.insert(flight + ',' + str(flightTreeIndex), (coordinates[0], coordinates[1], coordinates[0], coordinates[1]))
                flightTreeIndex+=1
        self.adsbTree = adsbTree
        
    def burnEncounterTrees(self):
        
        encounterResults = []

        for uasHeat in self.uasHeat:

            result = self.adsbTree.intersect(self.uasHeat[uasHeat])
            if len(result) > 0:

                encounterResults.append(result)
            
        self.encounterResults = encounterResults
        
        
        encounterFlights = []
        for uasEncounterList in self.encounterResults:
            for encounter in uasEncounterList:
                encounter = encounter.split(',')
                if encounter[0] in encounterFlights:
                    ;
                else:
                    encounterFlights.append(encounter[0])

        self.encounterFlights = encounterFlights
        ;

    def refreshEncounterDisplay(self):
        oldColors = []
        colorShuffle = 0
        for name in self.adsbFlights:

            if self.adsbLines[self.adsbFlights.index(name)].color == 'gold':
                self.adsbLines[self.adsbFlights.index(name)].color = self.oldColors[colorShuffle][0]
                colorShuffle +=1

            if name in self.encounterFlights:
                oldColors.append((self.adsbLines[self.adsbFlights.index(name)].color, self.adsbFlights.index(name)))
                self.adsbLines[self.adsbFlights.index(name)].color = 'gold'
            else:
                ;
        self.oldColors = oldColors

    def rebuildDf(self):
        
        
        self.heatDisplay.clear_layers()
        self.heatDisplay = None        
        self.uasHeat = {}
        self.adsbTree = {}
        self.encounterList = None
        self.buildAdsbFlights()
        self.refreshAdsbFlights()
        self.refreshAdsbDisplay()
        self.refreshUasFlights()
        self.refreshUasDisplay()
        self.refreshAdsbTree()
        self.refreshUasHeatTree(0)
        self.burnEncounterTrees()
        self.refreshEncounterDisplay()
        
        if self.mapp == None:
            mapp.add_layer(self.heatDisplay)
        else:
            self.mapp.add_layer(self.heatDisplay)
            

        

In [85]:
class mappConstructor():
    def __init__(self):
        
        self.mdf = None
        self.airport = 'DAB'
        self.mapp = None
        self.zoom = 10
        self.dataType = ['ADSB', 'AERO', 'TFMS']
        self.dataDate = ['2019', '08', '24']
        self.altitudeLow = 0
        self.altitudeHigh = 4000
        self.time = [' 00:00:00', ' 24:00:00']
        self.encounterDisplay = None
        self.adsbSelector = None
        
    def buildDataFrame(self):
            
        
        mdf = mappDataFrame()
        mdf.buildDataFrames(self.dataType, self.dataDate, self.airport)
        mdf.refreshParameters(self.altitudeLow ,self.altitudeHigh, self.time)
        mdf.buildAdsbFlights()
        mdf.refreshAdsbFlights()
        mdf.refreshAdsbDisplay()
        mdf.refreshUasFlights()
        mdf.refreshUasDisplay()
        mdf.refreshAdsbTree()
        mdf.refreshUasHeatTree(0)
        mdf.burnEncounterTrees()
        mdf.refreshEncounterDisplay()
        
        self.mdf = mdf
        
    def construct(self):
        
        
        
        mapp = Map(
    layers=(basemap_to_tiles(basemaps.Stamen.Toner), ),
        zoom = self.zoom,
        )
        
        mapp.add_layer(self.mdf.adsbDisplay)
        mapp.add_layer(self.mdf.uasDisplay)
        mapp.add_layer(self.mdf.heatDisplay)
        mapp.add_control(FullScreenControl()) 
        
        if self.airport == 'DAB':
            mapp.center = [29.1832, -81.0532]
            
        elif self.airport == 'ABQ':
            mapp.center = [35.0433, -106.6129]
            
        self.mapp = mapp
        self.addWidgets()
        self.mdf.mapp = self.mapp
        
    def display(self):
        
        return self.mapp
    
    def rebuildMdf(self, altitudeLow, altitudeHigh, timeLow, timeHigh):
    
        self.altitudeLow = altitudeLow
        self.altitudeHigh = altitudeHigh
        self.time = [timeLow, timeHigh]
        
        
        
        self.mdf.refreshParameters(self.altitudeLow, self.altitudeHigh, self.time)
        self.mdf.rebuildDf()
    
    def refreshEncounterDistance(self, heatDistanceDelimiter):
    
        self.mdf.refreshUasHeatTree(heatDistanceDelimiter)
        self.mdf.burnEncounterTrees()
        self.mdf.refreshEncounterDisplay()

        
    def addWidgets(self):
        
        encounterDisplay = widgets.BoundedIntText(
            value = 0,
            min = 0,
            max = 3000,
            step = 20,
            description = 'encounterDistance',
            disabled = False,
            continuous_update=True
        )

        encounterOutput = widgets.Output()

        def encounterChange(change):
            with encounterOutput:
                self.refreshEncounterDistance(encounterDisplay.value)

        
        encounterDisplay.observe(encounterChange, names='value')
        self.encounterDisplay = encounterDisplay
        
        adsbSelector = widgets.SelectMultiple(
            options = self.mdf.adsbFlights,
            value = self.mdf.adsbFlights[0:2],
            description = 'adsbFlights',
            continuous_update = True)

        adsbFlightOutput = widgets.Output()

        def adsbChange(change):
            with adsbFlightOutput:
                self.mdf.adsbDisplay.clear_layers()
                for x in adsbSelector.options:
                    if ( str(x) in adsbSelector.value ):
                        self.mdf.adsbDisplay.add_layer(
                            self.mdf.adsbLines[self.mdf.adsbFlights.index(x)])     
                    else:
                        ;

        adsbSelector.observe(adsbChange, names = 'value')
        self.adsbSelector = adsbSelector
        
        accordion = widgets.Accordion(children=[adsbSelector, encounterDisplay])
        accordion.set_title(0, 'adsb')
        accordion.set_title(1, 'encounter')
        
        widgetControl = WidgetControl(widget = accordion, position = 'bottomright')
        self.mapp.add_control(widgetControl)
    

In [86]:
mapp = mappConstructor()
mapp.buildDataFrame()

In [87]:
mapp.construct()

In [88]:
mapp.display()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [957]:
mapp.rebuildMdf(0, 2000, ' 06:00:00', ' 24:00:00')

In [68]:
mapp.refreshEncounterDistance(0)

In [20]:
list(mapp.mdf.uasFlights.keys())

['08RDE620010192',
 '08RDE3D00102QZ',
 '163DG1A001DPZJ',
 '08RDE6Q00102N9',
 '0K1CF5S3AE1EMT',
 '07JDEB600201L6',
 '163DF810013D1X',
 '163DFBM001PRNP',
 '163DFAC001ZQP0',
 '08RDE8G00104KD',
 '08RDEB600103TS',
 '11VKF2P00200S5',
 '08RDE1H00102Q0',
 '08RDEB400103ZM',
 '0M6DF6N00198G8',
 '0BMUFAB00100AF',
 '163DFBM001629Q',
 '08RDDC5001023K',
 '08RDE6100104DH',
 '0K1UF6600F0070',
 '163DF8J0011M92',
 '11VKF6P00202K6',
 '08RDE3T001002Z',
 '07JDDAJ00100S6',
 '163DG1Q00152MJ',
 '08RDEA300103T0',
 '08RDE8X001015W',
 '163DF8L001MME2',
 '03Z0017922',
 '08RDE8700100YS',
 '163DG1Q001LMYM',
 '08RDE8F001021K',
 '03Z1194928',
 '163DF8M0013L99',
 '03Z1072331',
 '163DFCP001N94W',
 '08RDDBF00100Q2',
 '0M6DF91001678J',
 '163DFA6001VBKR',
 '0BMUE8M00101P7',
 '163DFAB001C56H',
 '08RDE1700104WP',
 'nan',
 '276DFAF0017X1Z',
 '0BMLE7F30100VS',
 '0K1DEC92BC0033',
 '0M63G2R00SH037',
 '07JDE4R00A00DH',
 '08RDE7C00103LX',
 '0BMLE5301501QA',
 '08RDDCT00103YA',
 '08RDE7A00103S4',
 '08RDEA300101K6',
 '03Z0504514',
 